<center>
<img src="http://www.bigdive.eu/wp-content/uploads/2012/05/logoBIGDIVE-01.png">
</center>

---

# Deep Learning

## André Panisson

---

`conda install -c conda-forge tensorflow `  
`conda install -c conda-forge keras`

In [ ]:
%pylab inline

import tensorflow as tf
import keras

## Load the Dataset

This example is using the MNIST database of handwritten digits 
(http://yann.lecun.com/exdb/mnist/)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## Convolutional Network with Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

In [ ]:
X_train, Y_train = mnist.train.images, mnist.train.labels
X_test, Y_test = mnist.test.images, mnist.test.labels

In [ ]:
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

### Baseline model

In [ ]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(n_input, input_dim=n_input, kernel_initializer='normal', activation='relu'))
    model.add(Dense(n_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Reference for Keras optimization algorithms:   
https://keras.io/optimizers/

In [ ]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model

scores = model.evaluate(X_test, Y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

### Larger model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('th')

In [ ]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(30, (5, 5), padding='valid', input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# build the model
model = larger_model()
# Fit the model
model.fit(X_train.reshape(-1, 1, 28, 28), Y_train,
          validation_data=(X_test.reshape(-1, 1, 28, 28), Y_test),
          nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test.reshape(-1, 1, 28, 28), Y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

# Inception V3 model

Implementation of Google Inception V3 model for Keras.

<img src="https://4.bp.blogspot.com/-TMOLlkJBxms/Vt3HQXpE2cI/AAAAAAAAA8E/7X7XRFOY6Xo/s1600/image03.png">

**References**
- [Rethinking the Inception Architecture for Computer Vision](http://arxiv.org/abs/1512.00567)
- [Google Research Blog](https://research.googleblog.com/2016/03/train-your-own-image-classifier-with.html)

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.preprocessing import image

In [ ]:
model = InceptionV3(weights='imagenet')

In [ ]:
img_path = 'elephant.jpg'   # 'kitten.png'
img = image.load_img(img_path, target_size=(299, 299))
imshow(img)

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))